### Functions/code order:
1. `2-ukb_target_pipe`
* Generate targets and select features by utility.
2. `Link_semmed_cuis`
* NER linking + KG linkage/graph filtering. 
* Also contains code for future DB validation/checking (what appears in future). 
* By default, output only cases with 0 kg hits. 
* NOTE! We want to keep also the feature names from this.
* Also does filtering by semantic sim, and removes features with near exact match in kg (i.e if a feature's top entity is almost identical to the feature, and that entity has a kg match, then drop feature and "children" concepts). 
* 
Outputs concepts/UMLS entities associated with features. (And 0 KG hits) .
3. `search_pubmed`
* Literature search. Also by concept/entity. 
* +- Filter by KG distance, i.e distance 2 (1 hop between target and entity). 

4. LLM - run med_Rag and rank outputs. 
* Reformats feature + disease as prompt(s), ) 
* - can run local or api model. and runs rag then sends to LLM using `MedRAG` (with modified prompt template, code)
* Note - needs extracting into terms and doing RAG, +- custom search. **Slow. **

--------------------------------
`config` = contains settings (inputs/outputs etc') for a given target/disease
* e.g. some diseases we manually added synonyms. and also define search terms (for lit search, etc')

In [48]:
import os
import warnings
warnings.filterwarnings("ignore")
# warnings.filterwarnings("W036")
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)
import pandas as pd
from configs import * ## all configs
from util import *

%load_ext autoreload
%autoreload 2

In [47]:
# !pip install scispacy feature_engine biopython

In [3]:
from Link_semmed_cuis import  * #link_kg_concepts #*
from ukb_target_pipe  import * # make_target_df,model_features#*
from search_pubmed import  * #run_search_pubmed #*

In [4]:
Fast_Run = False
# RUN_PIPE = False
# SAVE_OUTPUT = True

# Fast_Run = False
RUN_PIPE = True
SAVE_OUTPUT = True
# OUTPUT_NAME= "localMini_chol.csv"
OUTPUT_NAME= "localMini_ast.csv"

In [5]:
# targets = ["celiac AND disease", "Coeliac", "gluten AND allergy","gluten AND enteropathy"] # celiac

# targets = ["Gout"] #,"Arthritis AND uratica"]
# targets = ["Multiple sclerosis"]
# targets = ["disc displacement", "disc degeneration", "disc disorder", "back AND pain", "spine disease","Spondylosis"]
# targets = ["oesophagus cancer"]
# targets = ["Retinal Vein Occlusion", "Central retinal artery occlusion"]

# OUTPUT_RES_PREFIX = "gout_"
# OUTPUT_RES_PREFIX = "celiac_"
# OUTPUT_RES_PREFIX = "ms_"
# OUTPUT_RES_PREFIX = "spine_"
# OUTPUT_RES_PREFIX = "oesophagus_"
# OUTPUT_RES_PREFIX = "eye_occ_"


# # ###### Celiac:
# ## input:
# FEATURES_REPORT_PATH = "celiac_feature_report.csv"
# ## output path
# CANDIDATE_NOVEL_CUIS_FILEPATH = "candidate_novel_celiac.csv"
# TARGET_NAME = "Coeliac disease" # / Celiac
# additional_target_cui_terms_list = ["C5139492"] # gluten allergy
# # ######### Multiple Sclerosis (MS)
# FEATURES_REPORT_PATH = "MS_ipw_feature_report.csv"
# CANDIDATE_NOVEL_CUIS_FILEPATH = "candidate_novel_MS.csv"
# TARGET_NAME = "Multiple Sclerosis"
# additional_target_cui_terms_list = []
# # ######### Spine Degeneration
# FEATURES_REPORT_PATH = "spine_degen_feature_report.csv"
# CANDIDATE_NOVEL_CUIS_FILEPATH = "candidate_novel_spine.csv"
# TARGET_NAME = "Spine degeneration"
# additional_target_cui_terms_list = ["C0158266", "C0850918" , "C0021818", "C0038019", "C0158252","C0423673","C0024031"]
# # ######### Oesophagus cancer
# FEATURES_REPORT_PATH = "oesophagus_feature_report.csv"
# CANDIDATE_NOVEL_CUIS_FILEPATH = "candidate_novel_oesophagus.csv"
# TARGET_NAME = "Esophageal cancer"
# additional_target_cui_terms_list = []


# ## gout
# config = {"FEATURES_REPORT_PATH":"gout_ipw_feature_report.csv",
#           "CANDIDATE_NOVEL_CUIS_FILEPATH":"candidate_novel_cuis_gout.csv",
#           "TARGET_NAME": "Gout"}

# # heart attack
# config = {"FEATURES_REPORT_PATH":"heart_feature_report.csv",
#           "CANDIDATE_NOVEL_CUIS_FILEPATH":"candidate_novel_cuis_heart.csv",
#           "TARGET_NAME": "Heart attack"}

# #  Retinal Vein Occlusion. (Central retinal artery occlusion: H34.1 )
# # Note: Central retinal artery occlusion is a type of stroke and must be treated immediately.
# config = {"FEATURES_REPORT_PATH":"eye_occ_ipw_feature_report.csv",
#               "CANDIDATE_NOVEL_CUIS_FILEPATH":"candidate_novel_cuis_eye_occ.csv",
# "TARGET_NAME":"Retinal Vein Occlusion"
# # "additional_target_cui_terms_list" : []
# }

In [6]:
# # Example configuration dictionary - gallstones
config_gall = {
    "TARGET_CODES_LIST" :("K80", "K81", "K82"),  ## Cholelithiasis = gallstones 
    "do_IPW": True,
    "do_boruta_fs" : True,#False,
    "K_IPW_RATIO":9,
    'targets': [
        "Cholelithiasis",
        "Gallstone",
        "Gallbladder disease",
        "cholecystitis",
        "Cholangitis"
        ],
    "FEATURES_REPORT_PATH":"gallstone_ipw_broad_feature_report.csv",
    'QUERY_CANDIDATES_FILE': 'candidate_novel_cuis_chol.csv',
    'DO_MINI_COMBINED_PATH_FILT': False
    ,'SAVE_OUTPUTS': True,
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_cuis_chol.csv", # output path for features/util
    "TARGET_NAME" :"GALLSTONES, Cholelithiasis",
    "additional_target_cui_terms_list" : ["C0008325", "C0008311"],
    
    'OUTPUT_RES_PREFIX': 'gallstone_',
    # 'full_results_filename': 'candidates_search_results.csv', # OUTPUT_RES_PREFIX is added atop this 
    # 'filtered_results_filename': 'review_interesting_candidates_results.csv',# OUTPUT_RES_PREFIX is added atop this 
    # # 'cooc_count_filter_val': 10,  'SIGNIFICANT_PVAL': 0.3,
    # "DIAG_TIDY_TABLE_PATH" : "../../df_ukbb_aux_tidy.parquet", #"../ukbb-hack/df_diag_tidy.parquet",
    }


config_celiac = {
    "TARGET_CODES_LIST": ("K90",),
    "do_IPW": True,
    "do_boruta_fs": False,
    "K_IPW_RATIO": 9,
    "targets": ["celiac AND disease", "Coeliac", "gluten AND allergy", "gluten AND enteropathy"],
    "FEATURES_REPORT_PATH": "celiac_feature_report.csv",
    "QUERY_CANDIDATES_FILE": "candidate_novel_cuis_chol.csv",
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_celiac.csv",
    "TARGET_NAME": "Coeliac disease",
    "additional_target_cui_terms_list": ["C5139492"],
    "OUTPUT_RES_PREFIX": "celiac_",
}
config_gout = {
    "TARGET_CODES_LIST": ("M10", "M1A"),
    "do_IPW": True,
    "do_boruta_fs": False,
    "K_IPW_RATIO": 9,
    "targets": ["Gout"],
    "FEATURES_REPORT_PATH": "gout_feature_report.csv",
    "QUERY_CANDIDATES_FILE": "candidate_novel_cuis_chol.csv",
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_cuis_gout.csv",
    "TARGET_NAME": "Gout",
    "additional_target_cui_terms_list": [],
    "OUTPUT_RES_PREFIX": "gout_",
}
config_ms = {
    "TARGET_CODES_LIST": ("G35",),
    "do_IPW": True,
    "do_boruta_fs": False,
    "K_IPW_RATIO": 9,
    "targets": ["Multiple sclerosis"],
    "FEATURES_REPORT_PATH": "MS_feature_report.csv",
    "QUERY_CANDIDATES_FILE": "candidate_novel_cuis_chol.csv",
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_MS.csv",
    "TARGET_NAME": "Multiple Sclerosis",
    "additional_target_cui_terms_list": [],
    "OUTPUT_RES_PREFIX": "ms_",
}
config_spine = {
    "TARGET_CODES_LIST": ("M51.3", "M51.2", "M51.0", "M51.1", "M51.3", "M47"),
    "do_IPW": True,
    "do_boruta_fs": False,
    "K_IPW_RATIO": 9,
    "targets": ["disc displacement", "disc degeneration", "disc disorder", "back AND pain", "spine disease", "Spondylosis"],
    "FEATURES_REPORT_PATH": "spine_feature_report.csv",
    "QUERY_CANDIDATES_FILE": "candidate_novel_cuis_chol.csv",
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_spine.csv",
    "TARGET_NAME": "Spine degeneration",
    "additional_target_cui_terms_list": ["C0158266", "C0850918", "C0021818", "C0038019", "C0158252", "C0423673", "C0024031"],
    "OUTPUT_RES_PREFIX": "spine_",
}
config_oesophagus = {
    "TARGET_CODES_LIST": ("C15",),
    "do_IPW": True,
    "do_boruta_fs": False,
    "K_IPW_RATIO": 9,
    "targets": ["oesophagus cancer"],
    "FEATURES_REPORT_PATH": "oesophagus_feature_report.csv",
    "QUERY_CANDIDATES_FILE": "candidate_novel_cuis_chol.csv",
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_oesophagus.csv",
    "TARGET_NAME": "Esophageal cancer",
    "additional_target_cui_terms_list": [],
    "OUTPUT_RES_PREFIX": "oesophagus_",
}
config_heart = {
    "TARGET_CODES_LIST": ("I21.9",),
    "do_IPW": True,
    "do_boruta_fs": False,
    "K_IPW_RATIO": 9,
    "targets": ["Heart attack"],
    "FEATURES_REPORT_PATH": "heart_feature_report.csv",
    "QUERY_CANDIDATES_FILE": "candidate_novel_cuis_chol.csv",
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_cuis_heart.csv",
    "TARGET_NAME": "Heart attack",
    "additional_target_cui_terms_list": [],
    "OUTPUT_RES_PREFIX": "heart_",
}
config_eye_occ = {
    "TARGET_CODES_LIST": ("H34",),
    "do_IPW": True,
    "do_boruta_fs": False,
    "K_IPW_RATIO": 9,
    "targets": ["Retinal Vein Occlusion", "Central retinal artery occlusion"],
    "FEATURES_REPORT_PATH": "eye_occ_feature_report.csv",
    "QUERY_CANDIDATES_FILE": "candidate_novel_cuis_chol.csv",
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_cuis_eye_occ.csv",
    "TARGET_NAME": "Retinal Vein Occlusion",
    "additional_target_cui_terms_list": [],
    "OUTPUT_RES_PREFIX": "eye_occ_",
}

config_depression = {
    "TARGET_CODES_LIST": ("F32","F33"),
    "do_IPW": False,
    "do_boruta_fs": True,
    "targets": ["Depression","Depressive disorder"],
    "FEATURES_REPORT_PATH": "depression_report.csv",
    "QUERY_CANDIDATES_FILE": "candidate_novel_cuis_depression.csv",
    "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_depression.csv",
    "TARGET_NAME": "Depression",
    "additional_target_cui_terms_list": [],
    "OUTPUT_RES_PREFIX": "depression_",
}


In [7]:
# List of all configurations
all_configs = [config_gall,config_celiac, config_gout, config_ms, config_spine, config_oesophagus,  config_heart, config_eye_occ, config_asthma]
all_configs = [config_asthma]
# all_configs =[config_depression] # not outputting results for search pubmed correctly? # Error processing Depression: [Errno 2] No such file or directory: 'candidate_novel_cuis_depression.csv'

for config in all_configs:
    if RUN_PIPE:
        try:
            print("Running pipeline for:", config["TARGET_NAME"])
            df = make_target_df(TARGET_CODES_LIST=config["TARGET_CODES_LIST"], FAST=Fast_Run)
            print("-------"*5, "\nTarget extracted\n", "-------"*8)
            print(df.shape)

            df.to_parquet(config["OUTPUT_RES_PREFIX"]+"_raw_df.parquet",index=False)
            
            if config.get("do_IPW", True):
                df = ipw_downsampling(df, K_IPW_RATIO=config.get("K_IPW_RATIO", 9))
            print(df.shape, "IPW downsampled")
            
            res_dict = model_features(df=df, FAST=Fast_Run, do_boruta_fs=config.get("do_boruta_fs", True), # True,
                                      SAVE_OUTPUT=True,
                                      FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"])
            
            link_kg_concepts(FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"], CANDIDATE_NOVEL_CUIS_FILEPATH=config["CANDIDATE_NOVEL_CUIS_FILEPATH"],
                             TARGET_NAME=config["TARGET_NAME"],
                             SAVE_OUTPUTS=True)
            
            # Run the search and analysis pipeline
            run_search_pubmed(config)

        except Exception as e:
            print(f"Error processing {config['TARGET_NAME']}: {e}")


After filtering KG min count
12915192
TARGET_NAME (For Entity-KG linking Depression
All ents (Depression,)
Depression C0011570

--------------------------

Name:  Depression
CUI: C0011570, Name: Mental Depression
Definition: Depressive states usually of moderate intensity in contrast with MAJOR DEPRESSIVE DISORDER present in neurotic and psychotic disorders.
TUI(s): T048
Aliases (abbreviated, total: 17): 
	 DEPRESSED STATE, depression (emotion), depression (disease), depression, Depression NOS, monopolar depression, DEPRESSION, Depression, Mental, Depressive state NOS, Depression, mental
CUI: C0011581, Name: Depressive disorder
Definition: An affective disorder manifested by either a dysphoric mood or loss of interest or pleasure in usual activities. The mood disturbance is prominent and relatively persistent.
TUI(s): T048
Aliases (abbreviated, total: 26): 
	 depression nos, Depressive disorders, depression, Depressive Neurosis, Depressive Disorder, Depressive disorder (disorder), Depr

/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 

815 - No Entity feats:['Vascular/heart problems diagnosed by doctor Stroke|High blood pressure', 'A09.9 - Gastroenteritis and colitis of unspecified origin', 'Alcohol intake frequency. One to three times a month', 'Mental health conditions ever diagnosed by a professional type', 'Home area population density - urban or rural Scotland - Large Urban Area', 'Job code at visit personnel', 'Medication for cholesterol, blood pressure, diabetes, or take exogenous hormones Cholesterol lowering medication|Blood pressure medication|Insulin|Hormone replacement therapy', "Suffer from 'nerves'", 'Internal derangement of knee', 'Mental health conditions ever diagnosed by a professional depression attention', 'Mental health conditions ever diagnosed by a professional attacks', 'Job code at visit surveyors', 'medication d3 tablet', 'Job code at visit goods', 'Eosinophill percentage', 'Other serious medical condition/disability diagnosed by doctor', 'Job code at visit managers', 'medication bricanyl 25

Calculating Paths: 100%|███████████████████████████████████████████████████████████████████████████| 563/563 [00:55<00:00, 10.08it/s]

(563, 5) df_path_lengths
(2306, 19) df_hits
candidate_novel_depression.csv
feature_name             1036
cui                       441
cui_nomenclature          441
cui_def                   441
raw_name                 1036
F.Split-Feature Split    1035
dtype: int64


,feature_name,cui,cui_nomenclature,cui_def,KG_Hits,sim_score,feature_level_min_kg_hits,feature_level_sum_kg_hits,feature_level_avg_kg_hits,feature_importance,...,MutualInfoTarget,raw_name,F.Split-Lift (y==1),F.Split-Support,F.Split-Feature Split,sim_score_target_feat,sim_score_target_cui,shortest_path_length,simple_path_length,norm_path_length
1,Home area population density - urban or rural ...,C0442529,Urban environment,Urban environment. A delineated geographic are...,0,0.319,0.0,1.0,0.2,0.183,...,0.003,Home area population density - urban or rural_...,1.04,367801,Home area population density - urban or rural_...,0.00,0.04,999,0,0.00
2,Home area population density - urban or rural ...,C0724128,Urban tradename,Urban tradename.,0,0.150,0.0,1.0,0.2,0.183,...,0.003,Home area population density - urban or rural_...,1.04,367801,Home area population density - urban or rural_...,0.00,0.04,99,0,0.00
3,Home area population density - urban or rural ...,C2700386,Urban - qualifier,"Urban - qualifier. Relating to, located in, or...",0,0.387,0.0,1.0,0.2,0.183,...,0.003,Home area population density - urban or rural_...,1.04,367801,Home area population density - urban or rural_...,0.00,0.04,99,0,0.00
4,Home area population density - urban or rural ...,C0549177,Large,Large. Of considerable or relatively great siz...,0,0.273,0.0,1.0,0.2,0.183,...,0.003,Home area population density - urban or rural_...,1.04,367801,Home area population density - urban or rural_...,0.00,0.04,999,0,0.00
5,Home area population density - urban or rural ...,C1416798,LARGE1 gene,LARGE1 gene.,0,0.184,0.0,1.0,0.2,0.183,...,0.003,Home area population density - urban or rural_...,1.04,367801,Home area population density - urban or rural_...,0.00,0.04,2,30,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,Job code at visit and maintenance,,,,0,1.000,0.0,0.0,0.0,0.000,...,0.001,Job code at visit__and maintenance,1.01,380015,Job code at visit__and maintenance < 0.14,-0.02,-0.00,2,68,0.20
2302,Pulse wave reflection index,,,,0,1.000,0.0,0.0,0.0,0.003,...,0.000,Pulse wave reflection index,1.41,415,Pulse wave reflection index >= 1.60,0.02,0.05,2,68,0.20
2303,Job code at visit representatives,,,,0,1.000,0.0,0.0,0.0,0.000,...,0.000,Job code at visit__representatives,1.00,385648,Job code at visit__representatives < 0.32,-0.06,0.00,2,68,0.20
2304,Job code at visit chartered secretaries,,,,0,1.000,0.0,0.0,0.0,0.000,...,0.001,Job code at visit__chartered secretaries,1.00,385072,Job code at visit__chartered secretaries < 0.20,-0.05,0.03,2,68,0.20


Targets: ['(Depression) OR (Depressive disorder)']
Error processing Depression: [Errno 2] No such file or directory: 'candidate_novel_cuis_depression.csv'


## Use MedRag - LLM for ranking of candidates
* 

In [7]:
# config = config_asthma

In [8]:
# picks , vs all final candidates
df_all = pd.read_csv(f"{config.get('OUTPUT_RES_PREFIX', '')}{config.get('full_results_filename', 'candidates_search_results.csv')}")
display(df_all)

# df_all.drop_duplicates("feature_name")

,Query,feature_name,cui,cui_def,KG_Hits,sim_score,feature_level_sum_kg_hits,feature_level_avg_kg_hits,feature_importance,p_val,...,simple_path_length,norm_path_length,Target,Query Count,Target Count,Co-occurrence Count,Cooccurrence P-Value,MI,min_feat_kgHit,feature_level_sum_cooc_hits
0,acceptability,Acceptability of each blow result (text) | False,C0814633,acceptability.,0,0.447,0.0,0.0,0.004,0.000,...,3,inf,asthma,570350,224834,3868,1.000,0.158,0.0,1.0
1,Acetoacetate,Acetoacetate,NaN,Acetoacetates. Salts and derivatives of acetoa...,0,0.860,0.0,0.0,0.003,0.573,...,0,0.00,asthma,5088,224834,4,0.000,-2.950,0.0,1.0
2,Acetoacetate measurement,Acetoacetate,C0392886,Acetoacetate measurement. The determination of...,0,0.665,0.0,0.0,0.003,0.573,...,70,inf,asthma,951,224834,1,0.021,-2.531,0.0,0.0
3,Acetoacetates,Acetoacetate,C0000988,Acetoacetates. Salts and derivatives of acetoa...,0,0.860,0.0,0.0,0.003,0.573,...,0,0.00,asthma,5088,224834,4,0.000,-2.950,0.0,1.0
4,Acute Enteritis of the Mouse Intestinal Tract,Gastritis and duodenitis,C1522057,Acute Enteritis of the Mouse Intestinal Tract.,0,0.499,1.0,0.3,0.004,0.000,...,792,6.77,asthma,250,224834,0,0.218,0.000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,NaN,medication cod liver,NaN,NaN,0,1.000,0.0,0.0,0.000,0.000,...,4,0.01,asthma,40195,224834,187,0.000,-0.385,NaN,NaN
726,NaN,Apolipoprotein A Blood biochemistry,NaN,NaN,0,1.000,0.0,0.0,0.010,0.000,...,4,0.01,asthma,40195,224834,187,0.000,-0.385,NaN,NaN
727,NaN,(non-UK origin) Country of Birth mauritius,NaN,NaN,0,1.000,0.0,0.0,0.000,0.011,...,4,0.01,asthma,40195,224834,187,0.000,-0.385,NaN,NaN
728,NaN,"Average total household income before tax 31,0...",NaN,NaN,0,1.000,0.0,0.0,0.000,0.000,...,4,0.01,asthma,40195,224834,187,0.000,-0.385,NaN,NaN


In [9]:
## all final candidates
df = pd.read_csv(f"{config.get('OUTPUT_RES_PREFIX', '')}{config.get('filtered_results_filename', 'review_interesting_candidates_results.csv')}")
print(df.shape[0],"# rows before dropping dupes by feat name, feat imp")
df = df.drop_duplicates("feature_name")

df = df.query("(p_val<0.2)|(MutualInfoTarget>0.003)")
display(df.dropna(axis=1,how="all"))

86 # rows before dropping dupes by feat name


,feature_name,raw_name,feature_importance,p_val,corr,MutualInfoTarget,Co-occurrence Count,F.Split-Lift (y==1),F.Split-Feature Split,Query Count,Target,sim_score,shortest_path_length,simple_path_length
0,Skin colour Fair,Skin colour_Fair,0.011,0.000,-0.017,0.047,0,1.07,Skin colour_Fair < 0.50,83,asthma,0.456,99,318
1,Acceptability of each blow result (text) | False,Acceptability of each blow result (text) | Arr...,0.004,0.000,-0.039,0.007,0,1.06,Acceptability of each blow result (text) | Arr...,306,asthma,0.227,2,529
2,Leg fat free mass,Leg fat-free mass (right),0.004,0.002,-0.005,0.002,0,1.09,Leg fat-free mass (right) < -1.40,618,asthma,0.413,2,0
3,"F32.9 - Depressive episode, unspecified","F32.9 - Depressive episode, unspecified",0.004,0.122,0.000,0.001,0,1.61,"-16.00 <= F32.9 - Depressive episode, unspecif...",50,asthma,0.636,2,14
4,Gastritis and duodenitis,Gastritis and duodenitis,0.004,0.000,-0.008,0.000,0,1.53,-6.90 <= Gastritis and duodenitis < -4.60,250,asthma,0.499,2,792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Weight change during worst episode of depressi...,Weight change during worst episode of depressi...,0.004,0.009,-0.005,0.001,3165,1.00,Weight change during worst episode of depressi...,656077,asthma,1.000,2,4
82,Other ill-defined and unknown causes of morbid...,Other ill-defined and unknown causes of morbid...,0.004,0.000,-0.011,0.000,5858,1.46,-4.70 <= Other ill-defined and unknown causes ...,646517,asthma,0.254,99,0
83,(AST) asthma genetic risk,(AST) asthma genetic risk,0.100,0.000,0.106,0.005,6086,2.06,(AST) asthma genetic risk >= 2.00,471633,asthma,0.682,2,721
84,(ISS) ischaemic stroke genetic risk,(ISS) ischaemic stroke genetic risk,0.006,0.269,0.002,0.000,6086,1.58,-1.40 <= (ISS) ischaemic stroke genetic risk <...,471633,asthma,0.619,2,721


In [10]:
df.nunique()

feature_name             86
raw_name                 86
feature_importance       21
p_val                    27
corr                     43
MutualInfoTarget         24
Co-occurrence Count      26
F.Split-Lift (y==1)      50
F.Split-Feature Split    86
Query Count              80
Target                    1
sim_score                71
shortest_path_length      2
simple_path_length       48
Novel/Interesting?        0
Makes Sense?              0
COMMENTS                  0
dtype: int64

In [49]:
import sys
# sys.path.append(r'D:\Research2\MedRAG')
# sys.path.append(r'D:\Research2\MedRAG\src')
sys.path.append('/mnt/d/Research2/MedRAG/')
sys.path.append('/mnt/d/Research2/MedRAG/src')

## change to path with medrag and it's downloaded corpus!
os.chdir('/mnt/d/Research2/MedRAG/')

from src.medrag import MedRAG


sys.path.append('/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag/')
sys.path.append('/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag/src')
sys.path.append('/mnt/d/Research2/MedRAG/')
## change to path with medrag and it's downloaded corpus!
# os.chdir('/mnt/d/Research2/MedRAG/')
# os.chdir('/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag')
from src.medrag import MedRAG
from tqdm import tqdm
import json

from util import get_predictions_from_medrag

# CORPUS_DIR_CACHE_PATH = '/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag/corpus'
CORPUS_DIR_CACHE_PATH = '/mnt/d/Research2/MedRAG/corpus'

In [50]:
def generate_medrag_prompts(data):
    results = []
    for index, row in data.iterrows():
        # Clean and prepare data
        feature_name = row['feature_name']
        raw_name = row['raw_name']
        target = row['Target']
        # Clean target string (keeping 'OR' in)
        target_clean = target.replace('(', '').replace(')', '').strip()
        # Clean feature names
        feature_name_clean = feature_name.replace('_', ' ').strip()
        raw_name_clean = raw_name.strip()
        
        # Extract relevant metadata
        p_val = row['p_val']
        correlation = row['corr']
        feature_importance = row['feature_importance']
        sim_score = row.get('sim_score', 'N/A')
        query_count = row.get('Query Count', 'N/A')
        mutual_info = row.get('MutualInfoTarget', 'N/A')
        co_occurrence_count = row.get('Co-occurrence Count', 'N/A')
        shortest_path_length = row.get('shortest_path_length', 'N/A')
        simple_path_length = row.get('simple_path_length', 'N/A')
        
        # Determine direction of effect
        if correlation > 0:
            direction = 'positive'
        elif correlation < 0:
            direction = 'negative'
        else:
            direction = 'neutral'
        
        # Prepare context (relevant documents)
        context = (
            f"Feature Name: {feature_name_clean}\n"
            f"Raw Feature Name: {raw_name_clean}\n"
            f"Target Disease(s): {target_clean}\n"
            f"Correlation: {direction} relationship with the target disease\n"
            f"P-Value: {p_val}\n"
            f"Feature Importance: {feature_importance}\n"
            f"Mutual Information with Target: {mutual_info}\n"
            f"Similarity Score: {sim_score}\n"
            f"Query Count (related publications): {query_count}\n"
            f"Co-occurrence Count: {co_occurrence_count}\n"
            f"Shortest Path Length in Knowledge Graph: {shortest_path_length}\n"
            f"Simple Path Length: {simple_path_length}\n"
        )
        
        # Adjusted Novelty Question and Options
        novelty_question = (
            f"Is the association between the feature '{feature_name_clean}' (raw: '{raw_name_clean}') "
            f"and '{target_clean}' novel, surprising, not documented in current knowledge or contradicts established understanding?? "
            # f"It has a correlation of {correlation} (indicating a {direction} relationship with the target disease). "
         f"It has {direction} correlation with the disease. "
            # f"Does this feature provide new insights or contradict established understanding?"
        )
        novelty_options = {
            "A": "Yes, it is novel, contradicts established understanding or provides new insight.",
            "B": "No, it is already known or does not provide new insights."
        }
        
        # Adjusted Plausibility Question and Options
        plausibility_question = (
            f"Does it make sense for the feature '{feature_name_clean}' (raw: '{raw_name_clean}') to be associated with '{target_clean}' "
            f"based on known mechanisms or theories? Correlation is {direction} with the target disease. "
            f"Is there a plausible explanation for this relationship that makes sense?"
        )
        plausibility_options = {
            "A": "Yes, there is a plausible explanation for this relationship.",
            "B": "No, there is no plausible explanation for this relationship."
        }
        
        # Adjusted Utility Question and Options
        utility_question = (
            f"Assess the utility of the feature '{feature_name_clean}' (raw: '{raw_name_clean}') for predicting '{target_clean}'. "
            # f"With a p-value of {p_val}, correlation of {correlation}, and feature importance of {feature_importance}, "
            f"does this feature have practical relevance or meaningful potential utility?"
        )
        utility_options = {
            "A": "Yes, it has practical relevance or potential utility.",
            "B": "No, it lacks practical relevance or utility."
        }
        
        # Store the prompts in MedRAG format
        results.append({
            # 'feature': feature_name_clean,
            # 'raw_feature': raw_name_clean,
           'feature':  row['feature_name'],
       'raw_feature': row['raw_name'],
            "target": row['Target'],
            'context': context,
            'novelty_question': novelty_question,
            'novelty_options': novelty_options,
            'plausibility_question': plausibility_question,
            'plausibility_options': plausibility_options,
            'utility_question': utility_question,
            'utility_options': utility_options
        })
        
    return results


In [51]:
results = generate_medrag_prompts(df)
if Fast_Run:
    results = results[0:2]
results

[{'feature': 'Skin colour Fair',
  'raw_feature': 'Skin colour_Fair',
  'target': 'asthma',
  'context': 'Feature Name: Skin colour Fair\nRaw Feature Name: Skin colour_Fair\nTarget Disease(s): asthma\nCorrelation: negative relationship with the target disease\nP-Value: 0.0\nFeature Importance: 0.011\nMutual Information with Target: 0.047\nSimilarity Score: 0.456\nQuery Count (related publications): 83\nCo-occurrence Count: 0\nShortest Path Length in Knowledge Graph: 99\nSimple Path Length: 318\n',
  'novelty_question': "Is the association between the feature 'Skin colour Fair' (raw: 'Skin colour_Fair') and 'asthma' novel, surprising, not documented in current knowledge or contradicts established understanding?? It has negative correlation with the disease. ",
  'novelty_options': {'A': 'Yes, it is novel, contradicts established understanding or provides new insight.',
   'B': 'No, it is already known or does not provide new insights.'},
  'plausibility_question': "Does it make sense fo

In [52]:
from src.medrag import MedRAG
import pandas as pd
from tqdm import tqdm
import re
from sklearn.metrics import classification_report, roc_auc_score

import json

In [53]:
# LL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct" # works, less stable than 3.0 model?
# LL_NAME ="meta-llama/Llama-3.2-3B-Instruct"
# LL_NAME ="mistralai/Ministral-8B-Instruct-2410"
# LL_NAME ="meta-llama/Llama-3.2-3B-Instruct"
# LL_NAME ="meta-llama/Llama-3.2-1B-Instruct"
 # LL_NAME =  = "meta-llama/Meta-Llama-3-70B-Instruct" 
# LL_NAME = "aaditya/Llama3-OpenBioLLM-8B" # unsupported template
LL_NAME = "OpenScholar/Llama-3.1_OpenScholar-8B" 

# LL_NAME = "mistralai/Ministral-8B-Instruct-2410"
## OpenScholar/Llama-3.1_OpenScholar-8B  # openscholar moidel
# if Fast_Run:
#     LL_NAME ="meta-llama/Llama-3.2-1B-Instruct"
    # LL_NAME ="meta-llama/Llama-3.2-3B-Instruct"

### plausible

In [17]:
len(results)

86

In [54]:
# results[0]

In [55]:
import gc
import torch

del medrag
gc.collect()
torch.cuda.empty_cache()


In [56]:
medrag = MedRAG(llm_name=LL_NAME, rag=True,
                retriever_name="BM25" # "BM25" #"RRF-2"#"MedCPT" - RRF2 woudl take ~ 11 hours for pubmed
                 # retriever_name="RRF-2"
# , corpus_name= "MedCorp" 
, corpus_name= "MedText"# small
                ,HNSW=True
                ,corpus_cache = True
                ,db_dir= CORPUS_DIR_CACHE_PATH
               )

# medrag = MedRAG(llm_name=LL_NAME, rag=True,
#                 retriever_name= "RRF-2"#"BM25" # "BM25" #"RRF-2"#"MedCPT" - RRF2 woudl take ~ 11 hours for pubmed
# , corpus_name= "MedCorp" # small
#                 ,HNSW=True
#                 # ,cache = True
#                )

Initializing the document extracter...
Initialization finished!


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use cuda:0


In [20]:
%%time
plausible_pred = []
plausible_cot = []
plausible_answers_full = []
plausible_answers_json = []
answer_snippets = []
options = results[0]["plausibility_options"]

## big speedup: cache + load snippets/search results! TODO
# for index, row in tqdm(df2.iterrows(), total=df2.shape[0], desc="Processing rows"): # when using df
for index, row in enumerate(tqdm(results, total=len(results), desc="Processing rows")):
    question = row['plausibility_question']
    answer, snippets, scores = medrag.answer(question=question, options=options, k=6 if Fast_Run else 30) #32
    # Ensure the answer ends with a single closing curly brace
    if not answer.endswith("}"): ## what about " at end?
        if "}" not in answer:
            answer += "}" 
    
    # Use regex to replace two or more consecutive closing curly braces with just one
    answer = re.sub(r'\}+', '}', answer)
    
    try:
        # json_ans = json.loads(re.search('{.+}', answer, re.IGNORECASE).group(0))
        json_ans = json.loads(re.search(r'{.*?}', answer, re.DOTALL | re.IGNORECASE).group(0))
        
    except:
        json_ans = json.dumps({'step_by_step_thinking':None,'answer_choice':None})
        print("Failed parse")
        print(answer)

    try:
        cot = json_ans['step_by_step_thinking']
        pred = json_ans['answer_choice']
        print(cot)
    except:
        pred =answer.split('answer_choice')[-1] 
        cot = answer.split("step_by_step_thinking")[-1].split("answer_choice")[0]

    plausible_pred.append(pred)
    plausible_cot.append(cot)
    plausible_answers_full.append(answer)
    # plausible_answers_json.append(json_ans)
    answer_snippets.append(snippets)

Processing rows:   1%|▉                                                                               | 1/86 [00:19<27:26, 19.37s/it]

The relationship between skin color and asthma is not well understood, but there is evidence to suggest that people with fair skin may be more susceptible to asthma. One possible explanation is that fair skin has less melanin, which can make it more sensitive to UV radiation. UV radiation can trigger an allergic response in some people, leading to asthma symptoms. Additionally, fair skin may also be associated with other factors that increase the risk of asthma, such as a family history of the disease or exposure to environmental allergens. Therefore, there is a plausible explanation for the relationship between fair skin color and asthma.


Processing rows:   2%|█▊                                                                              | 2/86 [00:33<23:06, 16.51s/it]

The acceptability of each blow result (text) | False is related to the Jones dye test, which is used to assess the patency of the nasolacrimal duct. Asthma is a respiratory disease that affects the airways, not the eyes or the nasolacrimal duct. There is no known mechanism or theory that links the acceptability of each blow result (text) | False to asthma. Therefore, there is no plausible explanation for this relationship.


Processing rows:   3%|██▊                                                                             | 3/86 [00:48<21:30, 15.55s/it]

...

Asthma is a complex disease that involves both genetic and environmental factors. The genetic component of asthma is multifactorial, with multiple genes contributing to the disease. Research has identified several genetic variants associated with asthma risk, including those in genes encoding factors involved in the immune response, such as the IL-4 receptor. These genetic variants can increase the risk of developing asthma, particularly in individuals with a family history of the disease. Therefore, there is a plausible explanation for the relationship between asthma genetic risk and asthma.


Processing rows:  99%|██████████████████████████████████████████████████████████████████████████████ | 85/86 [23:59<00:14, 14.33s/it]

The relationship between (ISS) ischaemic stroke genetic risk and asthma is not directly related. Asthma is a chronic inflammatory disease of the airways, while ischaemic stroke is a condition caused by a blockage in the blood vessels supplying blood to the brain. There is no known mechanism or theory that links asthma to ischaemic stroke genetic risk. Therefore, there is no plausible explanation for this relationship.


Processing rows: 100%|███████████████████████████████████████████████████████████████████████████████| 86/86 [24:15<00:00, 16.93s/it]

The relationship between immigration to the UK and asthma is not directly related. However, the prevalence of asthma in the UK has been increasing since the early 1990s. The UK is a Westernized country, and the prevalence of asthma is higher in Westernized countries. The increase in asthma prevalence in the UK could be due to various factors, including environmental factors, genetic predisposition, and lifestyle changes. Immigration to the UK may not be a direct cause of asthma, but it could be a contributing factor to the increase in asthma prevalence in the UK. Therefore, there is a plausible explanation for this relationship.
CPU times: user 21min 20s, sys: 2min 57s, total: 24min 18s
Wall time: 24min 15s


In [34]:
### save snippets to save on retrieval

In [57]:
%%time
novel_pred = []
novel_cot = []
novel_answers_full = []

options = results[0]["novelty_options"]

# for index, row in tqdm(df2.iterrows(), total=df2.shape[0], desc="Processing rows"): # when using df
for index, row in enumerate(tqdm(results, total=len(results), desc="Processing rows")):
    question = row['novelty_question']
    answer, snippets, scores = medrag.answer(question=question, options=options,
                                              k=22
                                             # snippets=answer_snippets[index]
                                            )
    if not answer.endswith("}"): 
        if "}" not in answer:
            answer += "}" 
    
    # Use regex to replace two or more consecutive closing curly braces with just one
    answer = re.sub(r'\}+', '}', answer)
    
    try:
        # json_ans = json.loads(re.search('{.+}', answer, re.IGNORECASE).group(0))
        json_ans = json.loads(re.search(r'{.*?}', answer, re.DOTALL | re.IGNORECASE).group(0))
        
    except:
        json_ans = json.dumps({'step_by_step_thinking':None,'answer_choice':None})
        print("Failed parse")
        print(answer)

    try:
        cot = json_ans['step_by_step_thinking']
        pred = json_ans['answer_choice']
        print(cot)
    except:
        pred =answer.split('answer_choice')[-1] 
        cot = answer.split("step_by_step_thinking")[-1].split("answer_choice")[0]

    novel_pred.append(pred)
    novel_cot.append(cot)
    novel_answers_full.append(answer)

Processing rows:   1%|▉                                                                               | 1/86 [00:15<21:55, 15.47s/it]

The association between skin colour fair and asthma is not novel or surprising as it is already known that fair skin is more prone to skin cancer and other skin conditions due to increased exposure to UV radiation. However, the relationship between skin colour and asthma is not as well established. While there is some evidence to suggest that people with fair skin may be more susceptible to asthma, it is not a well-established correlation. Therefore, the association between skin colour fair and asthma is not novel, contradicts established understanding or provides new insight.


Processing rows:   2%|█▊                                                                              | 2/86 [00:29<20:37, 14.73s/it]

The association between the feature 'Acceptability of each blow result (text) | False' and 'asthma' is not novel, as it is already documented in the provided documents. The documents mention that false-positive results can occur in various tests, including the Jones dye test, and that false-negative results can also occur in tests such as the methacholine challenge test. Therefore, the association between the feature and asthma is not novel, surprising, or contradictory to established understanding.


Processing rows:   3%|██▊                                                                             | 3/86 [00:42<19:07, 13.83s/it]

...

The association between the feature 'Single episode of probable major depression' and 'asthma' is not documented in current knowledge. The feature 'Single episode of probable major depression' is a specifier for major depressive disorder, which is a mental health condition. Asthma is a respiratory condition. There is no established understanding of a direct association between these two conditions. Therefore, the association is novel and provides new insight.


Processing rows:  42%|█████████████████████████████████                                              | 36/86 [07:51<10:21, 12.42s/it]

The association between the feature 'Cholesterol in Small VLDL' and 'asthma' is not documented in current knowledge. The provided documents do not mention any relationship between VLDL and asthma. Therefore, the association is novel and contradicts established understanding.


Processing rows:  42%|█████████████████████████████████                                              | 36/86 [24:53<34:34, 41.48s/it]


KeyboardInterrupt: 

In [41]:
%%time
utility_pred = []
utility_cot = []
utility_answers_full = []

options = results[0]["utility_options"]

# for index, row in tqdm(df2.iterrows(), total=df2.shape[0], desc="Processing rows"): # when using df
for index, row in enumerate(tqdm(results, total=len(results), desc="Processing rows")):
    question = row['utility_question']
    answer, snippets, scores = medrag.answer(question=question, options=options,
                                             snippets=answer_snippets[index],
                                             k=8) #32
    if not answer.endswith("}"): ## what about " at end?
        if "}" not in answer:
            answer += "}" 
    
    # Use regex to replace two or more consecutive closing curly braces with just one
    answer = re.sub(r'\}+', '}', answer)
    
    try:
        # json_ans = json.loads(answer)
        # json_ans = json.loads(re.search('{.+}', answer, re.IGNORECASE).group(0))
        json_ans = json.loads(re.search(r'{.*?}', answer, re.DOTALL | re.IGNORECASE).group(0))
        
    except:
        json_ans = json.dumps({'step_by_step_thinking':None,'answer_choice':None})
        print("Failed parse")
        print(answer)

    try:
        cot = json_ans['step_by_step_thinking']
        pred = json_ans['answer_choice']
        print(cot)
    except:
        pred =answer.split('answer_choice')[-1] 
        cot = answer.split("step_by_step_thinking")[-1].split("answer_choice")[0]

    utility_pred.append(pred)
    utility_cot.append(cot)
    utility_answers_full.append(answer)

Processing rows:   0%|                                                                                        | 0/86 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [60]:
df_results_all = pd.DataFrame({'feature': [x['feature'] for x in results],
"target": [x['target'] for x in results],
"novel_pred" : novel_pred,
"novel_cot":novel_cot,
"novel_answer" : novel_answers_full,
"plausible_pred" : plausible_pred,
"plausible_cot" : plausible_cot,
"plausible_answer" : plausible_answers_full,
"utility_pred" : utility_pred,
"utility_cot" : utility_cot,
"utility_answer" : utility_answers_full,
       "snippets":answer_snippets
                              })

df_results_all

,feature,target,plausible_pred,plausible_cot
0,Skin colour Fair,asthma,A,The relationship between skin color and asthma...
1,Acceptability of each blow result (text) | False,asthma,B,The acceptability of each blow result (text) |...
2,Leg fat free mass,asthma,A,Asthma is a chronic inflammatory disease of th...
3,"F32.9 - Depressive episode, unspecified",asthma,B,The relationship between F32.9 - Depressive ep...
4,Gastritis and duodenitis,asthma,B,Gastritis and duodenitis are conditions that i...
...,...,...,...,...
81,Weight change during worst episode of depressi...,asthma,B,The relationship between weight change during ...
82,Other ill-defined and unknown causes of morbid...,asthma,A,The relationship between 'Other ill-defined an...
83,(AST) asthma genetic risk,asthma,A,Asthma is a complex disease that involves both...
84,(ISS) ischaemic stroke genetic risk,asthma,B,The relationship between (ISS) ischaemic strok...


In [64]:
# df_results_all[ "novel_pred"]= pd.Series(novel_pred)
# df_results_all[ "novel_cot"]= pd.Series(novel_cot)
# df_results_all["plausible_pred"] = df_results_all["plausible_pred"].str.replace("A","1").str.replace("B","0")
# df_results_all["novel_pred"] = df_results_all["novel_pred"].str.replace("A","1").str.replace("B","0")

df_results_all["plausible_pred"] = pd.to_numeric(df_results_all["plausible_pred"])
df_results_all["novel_pred"] = pd.to_numeric(df_results_all["novel_pred"])

In [82]:
df_results_all.select_dtypes("number").mean()
# df_results_all["novel_pred"].dropna().mean()

plausible_pred      0.630137
novel_pred          0.566667
p_val               0.008055
corr               -0.004055
MutualInfoTarget    0.011329
dtype: float64

In [ ]:
# if SAVE_OUTPUT:
#     df_results_all.to_csv(OUTPUT_NAME,index=False)


In [75]:
df2 = df.query("(p_val<0.15)|(MutualInfoTarget>0.001)")
df_results_all = df_results_all.merge(df2.set_index('feature_name')[['raw_name',  'p_val', 'corr',
       'MutualInfoTarget']],left_on="feature",right_index=True,how="inner")

df_results_all.sort_values(["plausible_pred","novel_pred","MutualInfoTarget"],ascending=False,inplace=True)
df_results_all

,feature,target,plausible_pred,plausible_cot,novel_pred,novel_cot,raw_name,p_val,corr,MutualInfoTarget
17,Fractured/broken bones in last 5 years,asthma,1,Fractured/broken bones in last 5 years is a ri...,1.0,The association between the feature 'Fractured...,Fractured/broken bones in last 5 years_No,0.000,-0.014,0.039
19,Stiffness method Direct entry,asthma,1,Asthma is a disease that affects the airways a...,1.0,The association between the feature 'Stiffness...,Stiffness method_Direct entry,0.000,0.012,0.017
2,Leg fat free mass,asthma,1,Asthma is a chronic inflammatory disease of th...,1.0,The association between leg fat free mass and ...,Leg fat-free mass (right),0.002,-0.005,0.002
10,HBA1C DF glycated haemoglobin genetic risk,asthma,1,Asthma is a complex disease with multiple risk...,1.0,The association between HBA1C DF glycated haem...,(HBA1C_DF) glycated haemoglobin genetic risk,0.000,0.008,0.001
11,Frequency of depressed mood in last 2 weeks,asthma,1,The relationship between asthma and depression...,1.0,The association between the feature 'Frequency...,Frequency of depressed mood in last 2 weeks_Do...,0.000,0.007,0.001
...,...,...,...,...,...,...,...,...,...,...
71,Leg pain when standing still or sitting,asthma,0,The relationship between leg pain when standin...,NaN,NaN,Leg pain when standing still or sitting_Yes,0.000,0.033,0.001
81,Weight change during worst episode of depressi...,asthma,0,The relationship between weight change during ...,NaN,NaN,Weight change during worst episode of depressi...,0.009,-0.005,0.001
40,SCZ schizophrenia genetic risk,asthma,0,The relationship between SCZ schizophrenia gen...,NaN,NaN,(SCZ) schizophrenia genetic risk,0.000,0.010,0.000
46,Osteoarthrosis localized primary,asthma,0,Osteoarthrosis is a degenerative joint disease...,NaN,NaN,"Osteoarthrosis, localized, primary",0.000,-0.006,0.000


In [76]:
# df_results_all.to_csv("asthma_annotated_llm_pipe_v1.csv") ## output in medrag